# 处理数据

In [1]:
import pandas as pd
import numpy as np

In [2]:
names_label = ('uid', 'label')
names_voice = ('uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out')
names_sms = ('uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out')
names_wa = ('uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date')

读取训练集

In [3]:
uid_train = pd.read_csv("train/uid_train.txt",sep='\t', names = names_label, 
                        header=None, encoding='utf-8',index_col = False)
voice_train = pd.read_csv("train/voice_train.txt",sep='\t',names = names_voice, 
                          header=None, encoding='utf-8',index_col = False, 
                          )

sms_train = pd.read_csv("train/sms_train.txt",sep='\t',names = names_sms, 
                          header=None, encoding='utf-8',index_col = False,
                        dtype={'start_time':str})
wa_train = pd.read_csv("train/wa_train.txt",sep='\t',names = names_wa, 
                          header=None, encoding='utf-8',index_col = False,
                       dtype={'date':str})

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
voice_train.head()

,uid,opp_num,opp_head,opp_len,start_time,end_time,call_type,in_out
0,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115956,26120033,1,1
1,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115623,26115707,1,1
2,u0113,38D54642A237A11BB18455FC1E505292,132,11,26174233,26174321,1,1
3,u0113,38D54642A237A11BB18455FC1E505292,132,11,26070423,26070512,1,0
4,u3340,010A66F2AD42C48C44897A3DEC96A2A1,139,11,26201745,26201825,1,1


读取测试集

In [5]:
#  读取测试集_a
voice_test_a = pd.read_csv("TestA/voice_test_a.txt",sep='\t',names = names_voice, 
                          header=None, encoding='utf-8',index_col = False, 
                          )
sms_test_a = pd.read_csv("TestA/sms_test_a.txt",sep='\t',names = names_sms, 
                          header=None, encoding='utf-8',index_col = False,
                        dtype={'start_time':str})
wa_test_a = pd.read_csv("TestA/wa_test_a.txt",sep='\t',names = names_wa, 
                          header=None, encoding='utf-8',index_col = False,
                       dtype={'date':str})

In [6]:
#  读取测试集_b
voice_test_b = pd.read_csv("TestB/voice_test_b.txt",sep='\t',names = names_voice, 
                          header=None, encoding='utf-8',index_col = False, 
                          )

sms_test_b = pd.read_csv("TestB/sms_test_b.txt",sep='\t',names = names_sms, 
                          header=None, encoding='utf-8',index_col = False,
                        dtype={'start_time':str})
wa_test_b = pd.read_csv("TestB/wa_test_b.txt",sep='\t',names = names_wa, 
                          header=None, encoding='utf-8',index_col = False,
                       dtype={'date':str})

预测结果 格式

In [7]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test_b['uid'])})
uid_test.to_csv('TestB/uid_test_b.txt',index=None)

### 合并训练和测试数据

In [8]:
voice_train['time'] = voice_train['end_time'] - voice_train['start_time']
voice_test_a['time'] = voice_test_a['end_time'] - voice_test_a['start_time']
voice_test_b['time'] = voice_test_b['end_time'] - voice_test_b['start_time']

In [9]:
voice = pd.concat([voice_train,voice_test_a, voice_test_b],axis=0)
sms = pd.concat([sms_train,sms_test_a, voice_test_b],axis=0)
wa = pd.concat([wa_train,wa_test_a, wa_test_b],axis=0)

## 通话记录 数据处理

In [10]:
# 通话的不同号码的数目 以及 通话的数目
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg(
    {'unique_count': lambda x: len(pd.unique(x)),'count':'count'}
    ).add_prefix('voice_opp_num_').reset_index()

voice_opp_head=voice.groupby(['uid'])['opp_head'].agg(
    {'unique_count': lambda x: len(pd.unique(x))}
    ).add_prefix('voice_opp_head_').reset_index()

# 通话号码的长度
voice_opp_len=voice.groupby(['uid','opp_len'])['uid']. \
        count().unstack().add_prefix('voice_opp_len_'). \
        reset_index().fillna(0)

# 通话的类别 数目
voice_call_type = voice.groupby(['uid','call_type'])['uid']. \
        count().unstack().add_prefix('voice_call_type_'). \
        reset_index().fillna(0)

# 拨出 还是 接听
voice_in_out = voice.groupby(['uid','in_out'])['uid']. \
        count().unstack().add_prefix('voice_in_out_'). \
        reset_index().fillna(0)
        
voice_time_len = voice.groupby('uid')['time'].sum().reset_index().fillna(0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


## 短信记录 数据处理

In [11]:
# 发短信的不同号码； 发短信的条数
sms_opp_num = sms.groupby(['uid'])['opp_num']. \
    agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}). \
    add_prefix('sms_opp_num_').reset_index()


sms_opp_head = sms.groupby(['uid'])['opp_head']. \
    agg({'unique_count': lambda x: len(pd.unique(x))}). \
    add_prefix('sms_opp_head_').reset_index()

sms_opp_len = sms.groupby(['uid','opp_len'])['uid']. \
    count().unstack().add_prefix('sms_opp_len_'). \
    reset_index().fillna(0)

sms_in_out = sms.groupby(['uid','in_out'])['uid']. \
    count().unstack().add_prefix('sms_in_out_'). \
    reset_index().fillna(0)
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


## 网站/app记录 数据处理

In [12]:
wa_name = wa.groupby(['uid'])['wa_name']. \
    agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}). \
    add_prefix('wa_name_').reset_index()
    
visit_cnt = wa.groupby(['uid'])['visit_cnt']. \
    agg(['max','median','mean','sum']). \
    add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura']. \
    agg(['max','median','mean','sum']). \
    add_prefix('wa_visit_dura_').reset_index()

up_flow = wa.groupby(['uid'])['up_flow']. \
    agg(['max','median','mean','sum']). \
    add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow']. \
    agg(['max','median','mean','sum']). \
    add_prefix('wa_down_flow_').reset_index()
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


## 构建特征

In [13]:
feature = [voice_time_len, voice_opp_len, voice_opp_num, voice_opp_head,
           voice_call_type, voice_in_out, 
           sms_opp_num, sms_opp_head, sms_opp_len, sms_in_out,
           wa_name,visit_cnt,visit_dura,
           up_flow, down_flow]

In [14]:
train_feature = uid_train
for feat in feature:
    train_feature = pd.merge(train_feature, feat, how = 'left', on = 'uid')

In [15]:
test_feature = uid_test
for feat in feature:
    test_feature = pd.merge(test_feature, feat, how='left', on='uid')

In [16]:
train_feature.to_csv('train_feature_b.csv', index = None)
test_feature.to_csv('test_feature_b.csv', index = None)


In [17]:
test_feature.describe()

,time,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,voice_opp_len_9,voice_opp_len_10,voice_opp_len_11,voice_opp_len_12,...,wa_visit_dura_mean,wa_visit_dura_sum,wa_up_flow_max,wa_up_flow_median,wa_up_flow_mean,wa_up_flow_sum,wa_down_flow_max,wa_down_flow_median,wa_down_flow_mean,wa_down_flow_sum
count,2.992000e+03,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,...,2977.000000,3.000000e+03,2.977000e+03,2977.000000,2.977000e+03,3.000000e+03,2.977000e+03,2977.000000,2.977000e+03,3.000000e+03
mean,7.794657e+04,0.049131,1.553810,0.065174,0.328543,1.070187,0.144051,0.463570,195.162767,4.642714,...,36643.068552,4.778568e+07,3.188589e+07,5246.950286,1.855517e+05,2.632265e+08,3.878085e+08,9764.641753,2.249142e+06,3.205580e+09
std,2.271772e+05,0.331041,3.298143,0.381927,2.284075,4.388721,0.758087,2.588715,341.786708,25.529060,...,33408.548278,6.727552e+07,9.805226e+07,4891.713406,4.742723e+05,1.090643e+09,7.975682e+08,13383.024170,4.203855e+06,7.140193e+09
min,2.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,4.100000e+01,0.000000,3.725000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
25%,5.535000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,0.000000,...,13608.902985,2.490516e+06,1.061996e+06,2282.000000,3.045391e+04,5.547290e+06,9.855527e+06,2921.000000,1.883873e+05,3.333498e+07
50%,2.173350e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.500000,1.000000,...,30250.506767,2.100730e+07,8.005793e+06,4484.000000,9.305863e+04,6.918258e+07,9.776729e+07,7219.000000,7.898581e+05,6.314208e+08
75%,6.314975e+04,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,208.000000,3.000000,...,51143.398262,6.645744e+07,3.012408e+07,6955.000000,2.179306e+05,2.684254e+08,4.034042e+08,12938.000000,2.340189e+06,2.637244e+09
max,3.694800e+06,6.000000,49.000000,11.000000,67.000000,132.000000,22.000000,112.000000,4070.000000,1188.000000,...,511629.112821,5.684218e+08,3.171419e+09,71924.000000,1.857626e+07,5.310953e+10,1.435587e+10,422241.000000,5.358636e+07,9.303772e+10
